In [ ]:
'''Author: Masoud Rahimi, June 2023'''

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from scipy import stats
from numpy.linalg import norm

from sklearn.svm import SVC  
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

## Late Fusion

### Cross-validation

#### k-Fold cross-validation

In [2]:
def k_fold(id_sbjcts, k):
    ''' It gets a list of subjects' id and provides k sets of (id_train, id_test) data.
    input
        id_sbjcts[list]: it contains id of the subjects
        k: number of folds in Kfold. 
    output
        k_dataset[tuple]: it contains k tuples like this: (id_train, id_test)'''
    
    # determining the number of data points in the test and train sets
    n_test = int(len(id_sbjcts)//k)
    n_train = int(len(id_sbjcts) - n_test)
    k_dataset = []
    lower = 0
    for _ in range(k):
        id_test = id_sbjcts[lower:lower+n_test]
        id_train = [o for o in id_sbjcts if o not in id_test]
        #TODO: change the tuple to set          # not so important
        
        k_dataset.append((tuple(id_train), tuple(id_test)))
        lower += n_test
    return k_dataset

# # test the function output
# id_sbjcts = list(df_lightwheel['id'])
# for cnt, i in enumerate(k_fold(id_sbjcts, 6)):
#     print(f'fold {cnt} of data: (id_train, id_test) -> {i}')

#### Stratified k-Fold cross-validation

#### Leave-p-out cross-validation

### Load data

In [3]:
# load parrot data
dict_parrot = np.load('parrot_data.npy', allow_pickle=True).item()
df_parrot = pd.DataFrame.from_dict(dict_parrot,).T
df_parrot = df_parrot.reset_index()
df_parrot.rename(columns={'index':'id'}, inplace=True)


# load lightwheel data
df_lightwheel = pd.read_csv('lightwheel_data.csv')


# load car data
df_car = pd.read_excel('car_data.xlsx')
df_car = df_car.T.reset_index()
df_car.rename(columns={'index':'id', 7: 'label'}, inplace=True)


# df_parrot

### Classes & functions

In [4]:
class ParrotClassifier:
    def __init__(self, df):
        ''' It classifies ASD and non-ASD subjects
        df: dataframe containing all train and test data'''
        assert type(df) == pd.core.frame.DataFrame, 'df should be a dataframe containing test and train data'
        self.df = df.astype({'id': 'int32'})


    def fit(self, id_train):
        '''fit the classifier forr the given train data. each subjects has three vector.  
        ### note: the data with id==121 is delted from training since he/she did not react to the parrot.
        id_train[tuple]: it determines the training data. As you know each row of data has an id.'''
        assert type(id_train) == tuple, 'id_train should be a tuple'
        #TODO (done): exclude subject with id==121
        self.df_train = self.df[self.df['id'].isin(set(id_train) - {121})].copy().sort_values('id')
        # calculate the mean value of each feature in each class for the training data
        self.df_mean_of_matrices = self.df_train.drop(columns='id').groupby('label').mean().reset_index()

        ### !!! IMPORTANT !!! 
        self.accuracy_train = self.score(id_train)
        self.prediction_train = self.prediction
        self.label_train = self.label

    @staticmethod
    def cosine_similarity(A, B):
        return np.dot(A,B)/(norm(A)*norm(B))


    def score(self, id_test):
        '''Return the accuracy of the given test data'''
        assert type(id_test) == tuple, 'id_test should be a tuple'
        self.df_test = self.df[self.df['id'].isin(id_test)].copy().sort_values('id')
        self.df_test.at[:, 'predict'] = np.nan
        
        ### determining similarities in order to make a decision
        columns = ['id_vector', 'main_vector', 'move_vector']
        for matrix_name in columns:
            # creating new columns
            self.df_test.at[:, f'{matrix_name}_{"similarity_ASD"}'] = np.nan
            self.df_test.at[:, f'{matrix_name}_{"similarity_non_ASD"}'] = np.nan
            self.df_test.at[:, f'{matrix_name}_{"predict"}'] = np.nan
            
            # Iterate over row of  the dataframe
            for ind in self.df_test.index:
                A = self.df_test.loc[ind, matrix_name]                                    # matrix of the subject
                B = self.df_mean_of_matrices.loc[self.df_mean_of_matrices['label'] == '1', matrix_name].to_numpy()[0]    # mean matrix of the ASD group
                C = self.df_mean_of_matrices.loc[self.df_mean_of_matrices['label'] == '0', matrix_name].to_numpy()[0]    # mean matrix of the non-ASD group

                try:
                    self.df_test.at[ind, f'{matrix_name}_{"similarity_ASD"}'] = self.cosine_similarity(A, B)
                    self.df_test.at[ind, f'{matrix_name}_{"similarity_non_ASD"}'] = self.cosine_similarity(A, C)

                    # deciding based on similarity of each vector to mean of the ASD and non-ASD groups (we have three vectors, id, main, move)
                    if self.cosine_similarity(A, B) > self.cosine_similarity(A, C):   # if more similar to ASD
                        self.df_test.at[ind, f'{matrix_name}_{"predict"}'] = 1
                    elif self.cosine_similarity(A, B) < self.cosine_similarity(A, C):
                        self.df_test.at[ind, f'{matrix_name}_{"predict"}'] = 0
                    else:
                        self.df_test.at[ind, f'{matrix_name}_{"predict"}'] = np.nan
                except:
                    pass
        # max pooling on the predict of id, main, and move
        self.df_test.loc[:, 'predict'] = self.df_test[['id_vector_predict', 'main_vector_predict', 'move_vector_predict']].mode(axis=1)[0]
        self.prediction = self.df_test['predict'].to_numpy(float)
        self.label = self.df_test['label'].to_numpy(float)
        self.accuracy = np.sum(self.prediction == self.label) / len(self.df_test)
        
        return self.accuracy

id_sbjcts = tuple(df_lightwheel['id'])

### Take all data as training 
# parrot_clcfr = ParrotClassifier(df_parrot)
# parrot_clcfr.fit(id_sbjcts)
# print(parrot_clcfr.score(id_sbjcts))
# print(parrot_clcfr.accuracy_train)
# print(parrot_clcfr.label)
# print(parrot_clcfr.prediction)
# parrot_clcfr.df_test

## kfold evaluation
# for id_train, id_test in k_fold(id_sbjcts, 4):
    # print(id_train, id_test)
    # parrot_clcfr = ParrotClassifier(df_parrot)
    # parrot_clcfr.fit(id_train)
    # print(f'label train \t\t{parrot_clcfr.label_train}')
    # print(f'prediction train \t{parrot_clcfr.prediction_train}')
    # print(f'accuracy train \t\t{parrot_clcfr.accuracy_train}')
    # print(f'accuracty test \t\t{parrot_clcfr.score(id_test)}')
    # print(f'label test \t\t{parrot_clcfr.label}')
    # print(f'prediction test \t{parrot_clcfr.prediction}')
    # print('-----')

In [5]:
class LightWheelClassifier:
    def __init__(self, df):
        ''' It classifies ASD and non-ASD subjects
        df: dataframe containing all train and test data'''
        assert type(df) == pd.core.frame.DataFrame, 'df should be a dataframe containing test and train data'
        self.df = df.astype({'id': 'int32'})
        self.prediction = None
        self.label = None
        self.accuracy = None

    def fit(self, id_train):
        '''fit the classifier forr the given train data. each subjects has three vector.
        id_train[tuple]: it determines the training data. As you know each row of data has an id.'''
        assert type(id_train) == tuple, 'id_train should be a tuple'
        self.df_train = self.df[self.df['id'].isin(id_train)].copy().sort_values('id')
        self.dict_median = self.df_train[['id', 'detected_percentage', 'label']].groupby('label').median().reset_index()['detected_percentage'].to_dict()

        ### !!! IMPORTANT !!! 
        self.accuracy_train = self.score(id_train)
        self.prediction_train = self.prediction
        self.label_train = self.label

    def classify_based_median(self, p):
        ''' It classify each data points in three classes (ASD, non-ASD, no decision). the classification is based on the distance of the data to the median of each group
        p[float]: the data point. it is an scalar
        dict_median[dict]: the key is the class label and the values are the median of each class'''

        d_of_two_medians = (self.dict_median[max(self.dict_median)] - self.dict_median[min(self.dict_median)])/5
        d = d_of_two_medians/5      # divide the distance between two median to 5 sector.  
        if p >=  self.dict_median[max(self.dict_median)]:    # if the point is bigger than the maximum median, then it is in class with bigger median.
            result = max(self.dict_median)
        elif p <= self.dict_median[min(self.dict_median)]:    # if the point is smaller than the minimum median, then it is in class with smaller median.
            result = min(self.dict_median)
        elif abs(p - self.dict_median[max(self.dict_median)]) <= 2*d_of_two_medians:
            result = max(self.dict_median)
        elif abs(p - self.dict_median[min(self.dict_median)]) <= 2*d_of_two_medians:
            result = min(self.dict_median)
        else:
            result = 2
        return result
    

    def score(self, id_test):
        '''Return the accuracy of the given test data'''
        assert type(id_test) == tuple, 'id_test should be a tuple'
        self.df_test = self.df[self.df['id'].isin(id_test)].copy().sort_values('id')
        self.df_test.at[:, 'predict'] = np.nan
        self.df_test['predict'] = self.df_test.apply(lambda row: self.classify_based_median(row['detected_percentage']), axis=1)
        self.prediction = self.df_test['predict'].to_numpy(float)
        self.label = self.df_test['label'].to_numpy(float)
        self.accuracy = np.sum(self.prediction == self.label) / len(self.df_test)
        
        return self.accuracy

id_sbjcts = tuple(df_lightwheel['id'])
## Take all data as training 
# lightwheel_clcfr = LightWheelClassifier(df_lightwheel)
# lightwheel_clcfr.fit(id_sbjcts)
# print('here, test and train data are the same')
# print(f' label test \t\t{lightwheel_clcfr.label}')
# print(f' prediction test \t{lightwheel_clcfr.prediction}')
# print(f' accuracy test \t\t{lightwheel_clcfr.score(id_sbjcts)}')

# print(10*'====')
# ### kfold evaluation
# for id_train, id_test in k_fold(id_sbjcts, 4):
#     # print(id_train, id_test)
#     lightwheel_clcfr = LightWheelClassifier(df_lightwheel)
#     lightwheel_clcfr.fit(id_train)
#     print(f'label train \t\t{lightwheel_clcfr.label_train}')
#     print(f'prediction train \t{lightwheel_clcfr.prediction_train}')
#     print(f'accuracy train \t\t{lightwheel_clcfr.accuracy_train}')
#     print(f'accuracty test \t\t{lightwheel_clcfr.score(id_test)}')
#     print(f'label test \t\t{lightwheel_clcfr.label}')
#     print(f'prediction test \t{lightwheel_clcfr.prediction}')
#     print(17*'-----')


In [6]:
class CarClassifier:
    def __init__(self, *, df, id_train, id_test):
        ''' It classifies ASD and non-ASD subjects. subject with id==108 ignored the car, so if training data contains this subject's data, we delete it in fitting, 
        then add it to the dataset of training with nan value, since i have to keep predict_train for all classifiers with same id. result of car classifier for this 
        subject is Nan (still not implemented).
        df: dataframe containing ->  train + test + other data
        id_train[list]: id of train subjects
        id_test[list]: id of test subjects'''
        assert type(df) == pd.core.frame.DataFrame, 'df should be a dataframe containing test and train data'
        assert type(id_train) == tuple, 'id_train should be a tuple'
        assert type(id_test) == tuple, 'id_test should be a tuple'
        self.df = df.astype({'id': 'int32'})
        #TODO(done): exclued id==108
        self.id_train = set(id_train) - {108}          # subject with id==108 ignored the car and its data should be delted based on Bijan's notes.
        self.id_test  = set(id_test)
        self.id_test_train = self.id_train.union(self.id_test)
        self.id_other = set(df_car['id'].to_numpy()) - self.id_test_train     # data that are neither in test nor in train datasets but there are in th df



    def fit(self, *args):
        '''first, normalize the data (standard normalization), then normalize the test data based on this fitted scaler in the score method. Finally, fit the classifier for the given train data'''

        id_train_for_car = self.id_other.union(self.id_train)                 # data that are in id_other and id_train                
        self.df_train = self.df[self.df['id'].isin(id_train_for_car)].copy().sort_values('id')
        self.svm_clsfr = SVC(kernel='linear')  
        # scale the features (standard normalization)
        self.scaler = ColumnTransformer([('somename', StandardScaler(), list(range(0, 7)))], remainder='drop')
        self.scaler = self.scaler.fit(self.df_train[list(range(0, 7))])
        self.svm_clsfr.fit(self.scaler.transform(self.df_train[list(range(0, 7))]), self.df_train['label'].to_numpy())

        ### provide information about the training data
        self.accuracy_train = self.svm_clsfr.score(self.scaler.transform(self.df_train[list(range(0, 7))]), self.df_train['label'].to_numpy())
        self.prediction_train = self.svm_clsfr.predict(self.scaler.transform(self.df_train[list(range(0, 7))]))
        self.label_train = self.df_train['label'].to_numpy(float)
        

    def score(self, *args):
        '''Return the accuracy of the given test data. also calculate the prediction of the test data'''
        self.df_test = self.df[self.df['id'].isin(self.id_test)].copy().sort_values('id')
        self.df_test['predict'] = self.svm_clsfr.predict(self.scaler.transform(self.df_test[list(range(0, 7))]))      # use the standard scaler fitted based on train data
        # change the prediction for id==108 to non-ASD, since we have no data from him
        try:
            self.df_test[self.df_test['id'] == 108] = 0 
        except:
            print('except')
            pass
        self.prediction = self.df_test['predict'].to_numpy(float)
        self.label = self.df_test['label'].to_numpy(float)
        self.accuracy = np.sum(self.prediction == self.label) / len(self.df_test)
        
        return self.accuracy

# id_sbjcts = tuple(df_lightwheel['id'])
### Take all data as training     
# car_clcfr = CarClassifier(df=df_car, id_train=id_sbjcts, id_test=id_sbjcts)
# car_clcfr.fit()
# print('here, test and train data are the same')
# print(f' accuracy test \t\t{car_clcfr.score()}')
# print(f' label test \t\t{car_clcfr.label}')
# print(f' prediction test \t{car_clcfr.prediction}')
# print(10*'====')

# ### kfold evaluation
# for id_train, id_test in k_fold(id_sbjcts, 4):
    # print(id_train, id_test)
    # car_clcfr = CarClassifier(df=df_car, id_train=id_sbjcts, id_test=id_sbjcts)
    # car_clcfr.fit()
    # print(f'label train \t\t{car_clcfr.label_train}')
    # print(f'prediction train \t{car_clcfr.prediction_train}')
    # print(f'accuracy train \t\t{car_clcfr.accuracy_train}')
    # print(f'accuracty test \t\t{car_clcfr.score()}')
    # print(f'label test \t\t{car_clcfr.label}')
    # print(f'prediction test \t{car_clcfr.prediction}')
#     print(17*'-----')

### Evaluation

In [13]:
l = []
i = 0 
K = 4
for id_train, id_test in k_fold(id_sbjcts, K):
    # Initialize classifiers
    classifiers = [
        (CarClassifier(df=df_car, id_train=id_train, id_test=id_test), 'label_car', 'p_car', 'acc_car'),
        (LightWheelClassifier(df_lightwheel), 'label_lightwheel', 'p_lightwheel', 'acc_lightwheel'),
        (ParrotClassifier(df_parrot), 'label_parrot', 'p_parrot', 'acc_parrot')]

    results = {}
    results = {'fold': i, 'id': id_test,}
    for classifier, label_key, predict_key, acc_key in classifiers:
        clf = classifier
        clf.fit(id_train)

        results[acc_key] = clf.score(id_test)
        results[label_key] = clf.label
        results[predict_key] = clf.prediction
    # determine max pooling for each fold, and determining the acc of max pooling   
    p_stacked = np.vstack((results['p_car'], results['p_lightwheel'], results['p_parrot']))
    results['p_max_pooling'] = stats.mode(p_stacked, keepdims=False)[0]
    results['acc_max_pooling'] = np.sum(results['label_car'] == results['p_max_pooling']) / len(results['label_car'])
    l.append(results)
    i += 1

df_result = pd.DataFrame(l)
# df_result = df_result.explode(['id', 'label_car', 'label_lightwheel', 'label_parrot', 'p_car', 'p_lightwheel', 'p_parrot', 'p_max_pooling']).reset_index(drop=True)
df_result = df_result[['fold', 'id', 'p_car', 'p_lightwheel', 'p_parrot', 'p_max_pooling', 'label_lightwheel', 'acc_max_pooling', 'acc_car',   'acc_lightwheel', 'acc_parrot']]
df_result

# handling non decision output of classifiers (1. prediction for id==10 in car is nan, 2. prediction for id==121 in parrot is nan, 3.prediction for id==95 in lightwheel is nan)

,fold,id,p_car,p_lightwheel,p_parrot,p_max_pooling,label_lightwheel,acc_max_pooling,acc_car,acc_lightwheel,acc_parrot
0,0,"(87, 89, 94, 95)","[0.0, 1.0, 0.0, 1.0]","[1.0, 0.0, 0.0, 2.0]","[1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 0.0, 1.0]","[1.0, 0.0, 0.0, 1.0]",0.75,0.5,0.75,0.50
1,1,"(100, 102, 106, 120)","[1.0, 0.0, 1.0, 0.0]","[1.0, 0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 1.0]","[1.0, 0.0, 1.0, 0.0]","[1.0, 0.0, 1.0, 0.0]",1.00,1.0,1.00,0.00
2,2,"(110, 112, 118, 108)","[0.0, 1.0, 1.0, 1.0]","[0.0, 1.0, 0.0, 1.0]","[1.0, 1.0, 0.0, 1.0]","[0.0, 1.0, 0.0, 1.0]","[0.0, 1.0, 1.0, 1.0]",0.75,1.0,0.75,0.50
3,3,"(114, 116, 121, 124)","[1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 1.0, 0.0]","[1.0, 1.0, nan, 0.0]","[1.0, 1.0, 1.0, 0.0]","[1.0, 1.0, 0.0, 0.0]",0.75,0.5,0.25,0.75


In [15]:
df_accuracy = df_result.groupby('fold').mean(['acc_max_pooling', 'acc_car', 'acc_lightwheel', 'acc_parrot'])
print(df_accuracy.mean())
df_accuracy

acc_max_pooling    0.8125
acc_car            0.7500
acc_lightwheel     0.6875
acc_parrot         0.4375
dtype: float64


,acc_max_pooling,acc_car,acc_lightwheel,acc_parrot
fold,,,,
0,0.75,0.5,0.75,0.50
1,1.00,1.0,1.00,0.00
2,0.75,1.0,0.75,0.50
3,0.75,0.5,0.25,0.75


#### below coll, show some information about the train data.

In [ ]:
l = []
l_train = []
i = 0 
for id_train, id_test in k_fold(id_sbjcts, 4):
    # Initialize classifiers
    classifiers = [
        (CarClassifier(df=df_car, id_train=id_train, id_test=id_test), 'label_car', 'p_car', 'acc_car', 'label_car_train', 'p_car_train', 'acc_car_train'),
        (LightWheelClassifier(df_lightwheel), 'label_lightwheel', 'p_lightwheel', 'acc_lightwheel', 'label_lightwheel_train', 'p_lightwheel_train', 'acc_lightwheel_train'),
        (ParrotClassifier(df_parrot), 'label_parrot', 'p_parrot', 'acc_parrot', 'label_key_train_', 'p_parrot_train', 'acc_parrot_train')]

    results = {'fold': i, 'id': id_test,}
    results_train = {'fold': i, 'id': id_test,}
    for classifier, label_key, predict_key, acc_key, label_key_train, predict_key_train, acc_key_train in classifiers:
        clf = classifier
        clf.fit(id_train)

        results[acc_key_train] = clf.accuracy_train
        results_train[acc_key_train] = clf.accuracy_train
        results_train[predict_key_train] = clf.prediction_train
        results_train[label_key_train] = clf.label_train
        results[acc_key] = clf.score(id_test)
        results[label_key] = clf.label
        results[predict_key] = clf.prediction
    # determine max pooling for each fold, and determining the acc of max pooling   
    p_stacked = np.vstack((results['p_car'], results['p_lightwheel'], results['p_parrot']))
    results['p_max_pooling'] = stats.mode(p_stacked, keepdims=True)[0].reshape((4, ))
    results['acc_max_pooling'] = np.sum(results['label_car'] == results['p_max_pooling']) / len(results['label_car'])

    # p_stacked = np.vstack((results['p_car_train'], results['p_lightwheel_train'], results['p_parrot_train']))
    # results_train['p_max_pooling_train'] = stats.mode(p_stacked, keepdims=True)[0].reshape((4, ))
    # results_train['acc_max_pooling_train'] = np.sum(results['label_car_train'] == results['p_max_pooling_train']) / len(results['label_car_train'])
 
    l.append(results)
    l_train.append(results_train)
    i += 1


df_result = pd.DataFrame(l)
df_result = df_result.explode(['id', 'label_car', 'label_lightwheel', 'label_parrot', 'p_car', 'p_lightwheel', 'p_parrot', 'p_max_pooling', 'p_car_train', 'p_lightwheel_train', 'p_parrot_train']).reset_index(drop=True)
df_result = df_result[['fold', 'id', 'p_car', 'p_lightwheel', 'p_parrot', 'p_max_pooling', 'label_lightwheel', 'acc_max_pooling', 'acc_car',   'acc_lightwheel', 'acc_parrot', 'acc_car_train', 'acc_lightwheel_train', 'acc_parrot_train']]

# df_result_train = pd.DataFrame(l_train)
# df_result_train = df_result_train.explode(['id', 'label_car_train', 'p_car_train', 'p_lightwheel_train', 'p_parrot_train']).reset_index(drop=True)
# df_result_train = df_result_train[['fold', 'id',  'p_car_train', 'p_lightwheel_train', 'p_parrot_train', 'p_max_pooling_train', 'label_car_train', 'acc_max_pooling_train', 'acc_car_train', 'acc_lightwheel_train', 'acc_parrot_train']]

In [161]:
df_accuracy = df_result_new_label.groupby('fold').mean(['acc_max_pooling', 'acc_car', 'acc_lightwheel', 'acc_parrot', 'acc_car_train', 
                                              'acc_lightwheel_train', 'acc_parrot_train'])
df_accuracy

,p_car,p_lightwheel,p_parrot,label_car,acc_max_pooling,acc_car,acc_lightwheel,acc_parrot,acc_car_train,acc_lightwheel_train,acc_parrot_train,weighted_mean_pool,p_weighted_mean_pool,t
fold,,,,,,,,,,,,,,
0,-1.0,-0.5,1.0,0.0,0.75,0.50,0.75,0.50,0.822222,0.666667,0.916667,-0.099307,-0.5,0.75
1,0.0,0.0,1.0,0.0,0.75,0.50,1.00,0.50,0.800000,0.666667,0.916667,0.384615,0.5,0.75
2,1.0,0.0,0.0,0.0,0.25,0.50,0.50,0.25,0.844444,0.750000,0.916667,0.336283,0.5,0.25
3,1.0,-0.5,0.0,0.5,1.00,0.75,0.50,0.75,0.822222,0.750000,0.833333,0.185912,0.5,1.00


In [40]:
df_accuracy.mean()


NameError: name 'df_accuracy' is not defined

In [39]:
(75+75+25+100+25)/4

75.0

#### Weighted mean pooling

In [159]:
# convert label of "0" to "-1" since i want to calcualte weighted mean pooling on result of three classifiers
df_result_new_label = df_result.copy(deep=True)
# replace non decision output with label of non ASD. (prediction of nan, and 2 are non decision)
df_result_new_label[['p_car', 'p_lightwheel', 'p_parrot', 'label_car']] = df_result_new_label[['p_car', 'p_lightwheel', 'p_parrot', 'label_car']].replace([0, np.nan, 2], -1)
# weighted_mean_pool = predict(classifier)*accuracy_train(classifier)
df_result_new_label['weighted_mean_pool'] = df_result_new_label.apply(lambda x: (x['p_car']*x['acc_car_train'] + x['p_lightwheel']*x['acc_lightwheel_train']+x['p_parrot']*x['acc_parrot_train'])/(x['acc_car_train'] + x['acc_lightwheel_train'] + x['acc_parrot_train']), axis=1)
# conver wighted_mean_pool to binary prediction of the class. wighted_mean_pool<0, then predtion is non-ASD, and if wighted_mean_pool>0, then it is ASD
df_result_new_label['p_weighted_mean_pool'] = df_result_new_label['weighted_mean_pool'].apply(lambda x: -1 if x<0 else 1 )
# "t" represents the status predition for each test data. If True, then our classifiers integrations could classify correctly.
df_result_new_label['t'] = df_result_new_label.p_weighted_mean_pool == df_result_new_label.label_car
df_result_new_label[['fold', 'p_car', 'p_lightwheel', 'p_parrot', 'label_car', 'p_max_pooling', 'p_weighted_mean_pool', 't', 'weighted_mean_pool', 'acc_max_pooling']]

,fold,id,p_car,p_lightwheel,p_parrot,p_max_pooling,label_car,acc_max_pooling,acc_car,acc_lightwheel,acc_parrot,acc_car_train,acc_lightwheel_train,acc_parrot_train
0,0,87,-1.0,1.0,1.0,1.0,1.0,0.75,0.50,0.75,0.50,0.822222,0.666667,0.916667
1,0,89,-1.0,-1.0,1.0,0.0,-1.0,0.75,0.50,0.75,0.50,0.822222,0.666667,0.916667
2,0,94,-1.0,-1.0,1.0,0.0,-1.0,0.75,0.50,0.75,0.50,0.822222,0.666667,0.916667
3,0,95,-1.0,-1.0,1.0,0.0,1.0,0.75,0.50,0.75,0.50,0.822222,0.666667,0.916667
4,1,100,-1.0,1.0,1.0,1.0,1.0,0.75,0.50,1.00,0.50,0.800000,0.666667,0.916667
5,1,102,-1.0,-1.0,1.0,0.0,-1.0,0.75,0.50,1.00,0.50,0.800000,0.666667,0.916667
6,1,106,1.0,1.0,1.0,1.0,1.0,0.75,0.50,1.00,0.50,0.800000,0.666667,0.916667
7,1,108,1.0,-1.0,1.0,1.0,-1.0,0.75,0.50,1.00,0.50,0.800000,0.666667,0.916667
8,2,110,1.0,1.0,1.0,1.0,1.0,0.25,0.50,0.50,0.25,0.844444,0.750000,0.916667
9,2,112,1.0,-1.0,-1.0,0.0,1.0,0.25,0.50,0.50,0.25,0.844444,0.750000,0.916667


In [ ]:
df_accuracy = df_result_new_label.groupby('fold').mean(['acc_max_pooling', 'acc_car', 'acc_lightwheel', 'acc_parrot', 'acc_car_train', 
                                              'acc_lightwheel_train', 'acc_parrot_train'])
df_accuracy